In [ ]:
# This is my current data cleaning WIP for the project'

In [61]:
# Date 11/09/2024
import pandas as pd
import sqlite3
import numpy as np

# Load the dataset using a context manager
file_path = 'salary_data_states.xlsx'
with pd.ExcelFile(file_path) as xls:
    df = pd.read_excel(xls)

# Create a SQLite connection
conn = sqlite3.connect(':memory:')
df.to_sql('salary_data', conn, index=False, if_exists='replace')

167278

In [63]:
# Remove NA values using SQL query with correct column names
query_remove_na = """
SELECT 
    *
FROM 
    salary_data
WHERE 
    PAID_WAGE_PER_YEAR IS NOT NULL AND WORK_STATE IS NOT NULL
"""

cleaned_df = pd.read_sql_query(query_remove_na, conn)

In [67]:
# Remove duplicates directly in pandas
cleaned_df.drop_duplicates(inplace=True)

In [71]:
# Handling Missing Values - Impute Missing Values with mean for numerical columns
for column in cleaned_df.select_dtypes(include=[np.number]).columns:
    cleaned_df[column] = cleaned_df[column].fillna(cleaned_df[column].mean())

In [73]:
# Correcting Inconsistencies - Standardize Formats (e.g., dates)
if 'CASE_RECEIVED_DATE' in cleaned_df.columns:
    cleaned_df['CASE_RECEIVED_DATE'] = pd.to_datetime(cleaned_df['CASE_RECEIVED_DATE'], errors='coerce')
if 'DECISION_DATE' in cleaned_df.columns:
    cleaned_df['DECISION_DATE'] = pd.to_datetime(cleaned_df['DECISION_DATE'], errors='coerce')

In [75]:
# Correcting Inconsistencies - Fix Typos and Inconsistent Labels (example for WORK_STATE)
cleaned_df['WORK_STATE'] = cleaned_df['WORK_STATE'].str.upper()

In [77]:
# Handling Outliers - Example: Cap the PAID_WAGE_PER_YEAR at the 99th percentile
cap_value = cleaned_df['PAID_WAGE_PER_YEAR'].quantile(0.99)
cleaned_df['PAID_WAGE_PER_YEAR'] = np.where(cleaned_df['PAID_WAGE_PER_YEAR'] > cap_value, cap_value, cleaned_df['PAID_WAGE_PER_YEAR'])

In [79]:
# Ensuring Data Types are Correct - Convert Data Types (example for PAID_WAGE_PER_YEAR)
cleaned_df['PAID_WAGE_PER_YEAR'] = cleaned_df['PAID_WAGE_PER_YEAR'].astype(float)

In [83]:
# Removing Irrelevant Data - Drop Unnecessary Columns (example: 'order')
if 'order' in cleaned_df.columns:
    cleaned_df = cleaned_df.drop(columns=['order'])

In [85]:
# Handling Inconsistent Data - Check for Logical Consistency (example: no negative wages)
cleaned_df = cleaned_df[cleaned_df['PAID_WAGE_PER_YEAR'] >= 0]

In [87]:
# Encoding Categorical Variables - Convert Categorical Data to Numerical (example: FULL_TIME_POSITION_Y_N)
cleaned_df['FULL_TIME_POSITION_Y_N'] = cleaned_df['FULL_TIME_POSITION_Y_N'].map({'Y': 1, 'N': 0})

In [89]:
# Save the cleaned data to a new Excel file
cleaned_file_path = 'cleaned_salary_data_states.xlsx'
cleaned_df.to_excel(cleaned_file_path, index=False)

In [90]:
print(f"Cleaned data has been saved to {cleaned_file_path}")

Cleaned data has been saved to cleaned_salary_data_states.xlsx
